In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leap-atmospheric-physics-ai-climsim/sample_submission.csv
/kaggle/input/leap-atmospheric-physics-ai-climsim/test_old.csv
/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv
/kaggle/input/leap-atmospheric-physics-ai-climsim/test.csv
/kaggle/input/leap-atmospheric-physics-ai-climsim/sample_submission_old.csv


Import Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:

from sklearn.preprocessing import StandardScaler

In [4]:
nrows = 10000  
data = pd.read_csv('/kaggle/input/leap-atmospheric-physics-ai-climsim/train.csv'
, nrows=nrows)

In [5]:
data.head()


,sample_id,state_t_0,state_t_1,state_t_2,state_t_3,state_t_4,state_t_5,state_t_6,state_t_7,state_t_8,...,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
0,train_0,213.806117,222.244454,229.259879,245.743959,258.337482,265.772467,263.978456,254.879872,243.946147,...,-6.132147e-07,1.401824e-06,0.0,349.564325,0.0,2.947672e-12,0.0,0.0,0.0,0.0
1,train_1,213.177430,225.851064,229.663618,246.828333,261.026415,269.078431,267.736565,256.896227,244.169421,...,3.560950e-07,9.984912e-07,0.0,335.204086,0.0,3.138366e-09,0.0,0.0,0.0,0.0
2,train_2,217.105685,220.448106,234.653398,244.422951,254.023818,259.651472,257.709514,251.064513,241.879600,...,1.565672e-06,6.186607e-05,0.0,401.709340,0.0,7.424215e-09,0.0,0.0,0.0,0.0
3,train_3,217.773994,225.611775,234.104091,247.745365,257.411402,263.470947,261.131775,253.303250,242.316814,...,-9.067304e-06,1.020519e-05,0.0,400.230177,0.0,2.534090e-08,0.0,0.0,0.0,0.0
4,train_4,216.349337,230.526083,233.650252,248.196013,262.500730,270.055663,268.863606,258.161645,244.442620,...,1.681596e-06,-6.378779e-06,0.0,321.960470,0.0,3.377415e-10,0.0,0.0,0.0,0.0


In [6]:
data = data.drop(columns='sample_id')
data = data.astype(np.float32)
data_values = data.values


In [7]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.values)

input_timesteps = 20  
output_timesteps = 10  
num_features = data_scaled.shape[1]

def create_sequences(data, in_steps, out_steps):
    X, y = [], []
    for i in range(len(data) - in_steps - out_steps + 1):
        X.append(data[i: i + in_steps])
        y.append(data[i + in_steps: i + in_steps + out_steps])
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled, input_timesteps, output_timesteps)
print("X shape:", X.shape)  
print("y shape:", y.shape) 

X shape: (9971, 20, 924)
y shape: (9971, 10, 924)


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense


latent_dim = 128


encoder_inputs = Input(shape=(input_timesteps, num_features), name='encoder_input')

encoder_lstm = LSTM(latent_dim, return_state=True, name='encoder_lstm')
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(output_timesteps, num_features), name='decoder_input')

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_features, activation='linear', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_input             │ (None, 20, 924)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_input             │ (None, 10, 924)        │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_lstm (LSTM)       │ [(None, 128), (None,   │        539,136 │ encoder_input[0][0]    │
│                           │ 128), (None, 128)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_lstm (LSTM)       │ [(None, 10, 128),      │        539,136 │ decoder_input[0][0],   │
│                           │ (None, 128), (None,    │                │ encoder_lstm[0][1],    │
│                           │ 128)]                  │                │ encoder_lstm[0][2]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dense (Dense)     │ (None, 10, 924)        │        119,196 │ decoder_lstm[0][0]     │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,197,468 (4.57 MB)

 Trainable params: 1,197,468 (4.57 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
decoder_input_data = np.zeros_like(y)
decoder_input_data[:, 1:, :] = y[:, :-1, :]
decoder_input_data[:, 0, :] = 0  

# Train the model
history = model.fit(
    [X, decoder_input_data],  
    y,                        
    batch_size=64,
    epochs=20,
    validation_split=0.2
)

Epoch 1/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 117ms/step - loss: 0.6392 - val_loss: 0.5432
Epoch 2/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 114ms/step - loss: 0.4215 - val_loss: 0.5089
Epoch 3/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 116ms/step - loss: 0.3507 - val_loss: 0.4903
Epoch 4/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 117ms/step - loss: 0.3148 - val_loss: 0.4827
Epoch 5/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 121ms/step - loss: 0.3003 - val_loss: 0.4795
Epoch 6/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 122ms/step - loss: 0.2813 - val_loss: 0.4763
Epoch 7/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 113ms/step - loss: 0.2652 - val_loss: 0.4740
Epoch 8/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 21s 117ms/step - loss: 0.2543 - val_loss: 0.4754
Epoch 9/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 110ms/step - loss: 0.2590 - val_loss: 0.4762
Epoch 10/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 114ms/step - loss: 0.2472 - val_loss: 0.4788
Epoch 11/20
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 110ms/step - loss: 0.2343 - val_loss: 0.4822
Epoch 12/20
125/125

In [10]:
# Inference Encoder: same as training encoder.
encoder_model = Model(encoder_inputs, encoder_states)

# Inference Decoder setup:
# New input placeholders for decoder states.
decoder_state_input_h = Input(shape=(latent_dim,), name='input_h')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# For inference, the decoder will take one timestep at a time.
decoder_single_input = Input(shape=(1, num_features), name='decoder_single_input')
decoder_outputs, state_h, state_c = decoder_lstm(decoder_single_input, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_single_output = decoder_dense(decoder_outputs)

# Define the inference decoder model.
decoder_model = Model(
    [decoder_single_input] + decoder_states_inputs,
    [decoder_single_output] + decoder_states
)

In [11]:
def decode_sequence(input_seq):
    # Encode the input to get the initial states.
    states_value = encoder_model.predict(input_seq)
    
    # Start with a zero vector as the first decoder input.
    target_seq = np.zeros((1, 1, num_features))
    decoded_seq = []
    
    # Generate the output sequence one timestep at a time.
    for _ in range(output_timesteps):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        decoded_seq.append(output_tokens[0, 0, :])
        
        # Update the target sequence to the current prediction.
        target_seq = output_tokens.reshape(1, 1, num_features)
        states_value = [h, c]
    
    return np.array(decoded_seq)

# Test the inference on the first sample.
predicted_sequence = decode_sequence(X[0:1])
print("Predicted sequence shape:", predicted_sequence.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted sequence shape: (10, 924)
